In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import random
import time
from tqdm import tqdm, trange

In [2]:
base_url = "https://opengov.seoul.go.kr/civilappeal?"
page_source = requests.get( base_url )
soup = bs(page_source.text, 'html.parser')

In [3]:
div_row = soup.select("div#accordion")[0]
div_row.find("h4")

<h4><a href="#civilapeal-2897888">[ 복지 ] 여성복지상담소 어디에 있나요?</a></h4>

In [4]:
soup.select("h4")

[<h4><a href="#civilapeal-2897888">[ 복지 ] 여성복지상담소 어디에 있나요?</a></h4>,
 <h4><a href="#civilapeal-2897889">[ 복지 ] 여성복지상담소는 모든 상담이 무료인가요?</a></h4>,
 <h4><a href="#civilapeal-2895764">[ 행정ㆍ기타 ] 진정민원 제출시 증빙서류를 받을수 있나요?</a></h4>,
 <h4><a href="#civilapeal-2898246">[ 재정ㆍ세금 ] 은평구에는 어떤종류의 하천이 있습니까 예를 들면 국가하천,지방하천 등</a></h4>,
 <h4><a href="#civilapeal-2897030">[ 행정ㆍ기타 ] [정보통신] 통신관련 민원처리기관과 통신사업자 민원처리 안내번호</a></h4>,
 <h4><a href="#civilapeal-2896772">[ 경제 ] 성동구치소 이전 ?</a></h4>,
 <h4><a href="#civilapeal-13865831">[ 문화ㆍ관광 ] [한강사업본부] 서울함 공원 (구.한강 함상공원)</a></h4>,
 <h4><a href="#civilapeal-2894946">[ 행정ㆍ기타 ] 국유지 매각기준을 보고 싶은데 어디서 볼수 있는지요?</a></h4>,
 <h4><a href="#civilapeal-2897733">[ 안전 ] 재해경보의 종류는?</a></h4>,
 <h4><a href="#civilapeal-2896853">[ 행정ㆍ기타 ] 시비 재배정사업 중 조경분야 서울시(계약심사과) 심사요청 대상은?</a></h4>,
 <h4><a href="#civilapeal-2895491">[ 경제 ] 창문에 썬팅을 하려고하는데 괜찮은가요?</a></h4>,
 <h4><a href="#civilapeal-2896850">[ 행정ㆍ기타 ] 계약심사 대상금액이란?</a></h4>,
 <h4><a href="#civilapeal-2896860">[ 행정ㆍ기타 ] 건축공사분야 실적공사비 적용 대상

In [5]:
content_step1 = soup.select("div.content-step1")[1]
content_step1

<div class="content-step1" id="civilappeal-2897889">
<p>- 네. 그렇습니다.    여성복지상담소는 금천구에서 직영하는 상담소로서    모든 상담이 무료로 진행되고 있습니다.</p>
<div class="list-tag">
<p class="tag"><em class="element-invisible">관련태그 :</em>사회보장과복지,여성복지,벧엘케어상담소</p>
<a class="link-orange" href="/civilappeal/2897889">상세보기  &gt;</a>
</div>
</div>

In [6]:
content_step1.find('p').get_text(strip=True)

'- 네. 그렇습니다.    여성복지상담소는 금천구에서 직영하는 상담소로서    모든 상담이 무료로 진행되고 있습니다.'

In [7]:
tag = soup.select("div.content-step1 > div.list-tag")[1].find('p').get_text(strip=True)
tag

'관련태그 :사회보장과복지,여성복지,벧엘케어상담소'

In [8]:
tag = tag.split("관련태그 :")[1]
tag

'사회보장과복지,여성복지,벧엘케어상담소'

In [9]:
# 제목, 내용, 태그
civilappeal = []
h4 = soup.select("h4")

for i, row in tqdm(enumerate(h4)):
    article = []
    no = row.find('a')['href'].split("-")[1]
    subject = row.find('a').get_text(strip=True)
    content = soup.select("div.content-step1")[i].find('p').get_text(strip=True)
    tag = soup.select("div.content-step1 > div.list-tag")[i].find('p').get_text(strip=True)
    tag = tag.split("관련태그 :")[1]
    article.append(no)
    article.append(subject)
    article.append(content)
    article.append(tag)
    civilappeal.append(article)

15it [00:00, 224.97it/s]


In [10]:
df_civilappeal = pd.DataFrame(civilappeal)
df_civilappeal.head()

,0,1,2,3
0,2897888,[ 복지 ] 여성복지상담소 어디에 있나요?,- 여성복지상담소는 독산1동 289-7번지 한빛빌딩 4층 자원봉사센터 내에 있...,"사회보장과복지,여성복지,벧엘케어상담소"
1,2897889,[ 복지 ] 여성복지상담소는 모든 상담이 무료인가요?,- 네. 그렇습니다. 여성복지상담소는 금천구에서 직영하는 상담소로서 모든...,"사회보장과복지,여성복지,벧엘케어상담소"
2,2895764,[ 행정ㆍ기타 ] 진정민원 제출시 증빙서류를 받을수 있나요?,- 진정민원 제출시 민원복지동 1층 민원여권과에서 접수증을 발부하여 드리고 있습니다.,"구정일반,민원관리,통합민원실안내"
3,2898246,"[ 재정ㆍ세금 ] 은평구에는 어떤종류의 하천이 있습니까 예를 들면 국가하천,지방하천 등","- 은평구는 국가하천은 없습니다. 지방하천으로 불광천,녹번천이 있고, 소하천으로 ...","세금과재정,재정,일상경비관리"
4,2897030,[ 행정ㆍ기타 ] [정보통신] 통신관련 민원처리기관과 통신사업자 민원처리 안내번호,- 업무개요 통신관련 민원 전화번호 정리 업무내용 ▣ 통신관...,"통신,정보자원관리,시스템및정보보호"


In [11]:
df_civilappeal.columns = ["번호", "제목", "내용", "태그"]
df_civilappeal.tail()

,번호,제목,내용,태그
10,2895491,[ 경제 ] 창문에 썬팅을 하려고하는데 괜찮은가요?,- 창문이용광고물(일명 썬팅)은 표시하여서는 안됩니다. 다만 건물의 1층 창문 또는...,"제조건설과개발,도시디자인,불법전단광고,벽보정비"
11,2896850,[ 행정ㆍ기타 ] 계약심사 대상금액이란?,- 도급비+관급비,"구정일반,자치행정,일반"
12,2896860,[ 행정ㆍ기타 ] 건축공사분야 실적공사비 적용 대상 범위는?,- 공사비 10억원이상,"구정일반,자치행정,일반"
13,2896846,[ 행정ㆍ기타 ] 공사분야 계약심사 요청 대상 범위는?,- 공사비 3000만원 이상,"구정일반,자치행정,일반"
14,2896852,[ 행정ㆍ기타 ] 시비 재배정사업 중 건축분야 서울시(계약심사과) 심사요청 대상은?,- 공사비 5억원이상,"구정일반,자치행정,일반"
